In [1]:
!pip install python-binance

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import binance_framework as bf
from collections import deque
import random

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
CURRENCIES = ['ETHUSDT']
SEQUENCE_LEN = 128
FUTURE_PERIOD_PREDICT = 5
RATIO_TO_PREDICT = 'ETHUSDT'
EPOCHS = 20
BATCH_SIZE = 128
NAME = 'SEQ-'+ str(SEQUENCE_LEN)+'-FUTPRED-'+str(FUTURE_PERIOD_PREDICT)+'-PRED-'+str(time.time())

In [0]:
#coins = bf.get_data_by_intervals(300,CURRENCIES,bf.INTERVAL_1MINUTE)

In [0]:
#coins.to_csv('eth_1min_300.csv',index=False)

In [0]:
coins  = pd.read_csv('eth_1min.csv')

In [8]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24951 entries, 0 to 24950
Data columns (total 12 columns):
ETHUSDT Open Time                       24951 non-null int64
ETHUSDT Open                            24951 non-null float64
ETHUSDT High                            24951 non-null float64
ETHUSDT Low                             24951 non-null float64
ETHUSDT Close                           24951 non-null float64
ETHUSDT Volume                          24951 non-null float64
ETHUSDT Close Time                      24951 non-null int64
ETHUSDT Quote asset volume              24951 non-null float64
ETHUSDT n_trades                        24951 non-null int64
ETHUSDT Taker buy base asset volume     24951 non-null float64
ETHUSDT Taker buy quote asset volume    24951 non-null float64
ETHUSDT Ignore                          24951 non-null int64
dtypes: float64(8), int64(4)
memory usage: 2.3 MB


In [9]:
coins.head()

,ETHUSDT Open Time,ETHUSDT Open,ETHUSDT High,ETHUSDT Low,ETHUSDT Close,ETHUSDT Volume,ETHUSDT Close Time,ETHUSDT Quote asset volume,ETHUSDT n_trades,ETHUSDT Taker buy base asset volume,ETHUSDT Taker buy quote asset volume,ETHUSDT Ignore
0,1581830340000,270.45,270.65,270.42,270.52,72.89036,1581830399999,19716.701573,73,46.70314,12632.706196,0
1,1581830400000,270.52,270.52,270.07,270.48,379.79188,1581830459999,102628.158319,222,208.31140,56284.419267,0
2,1581830460000,270.48,270.98,270.30,270.97,249.07819,1581830519999,67415.423924,191,131.22899,35520.299684,0
3,1581830520000,270.93,270.93,270.55,270.61,90.83813,1581830579999,24589.030344,109,22.07769,5976.571651,0
4,1581830580000,270.61,270.67,270.43,270.61,107.55053,1581830639999,29102.482234,72,29.49415,7980.919520,0


In [10]:
bf.show_time_skips(coins, CURRENCIES)

Nanosec default difference:  60000
ETHUSDT  Time difference:  21300000
between indexes:  4696 , 4697
gap starts at:  1582112100000
gap ends at:    1582133400000


ETHUSDT  Time difference:  7740000
between indexes:  24368 , 24369
gap starts at:  1583313660000
gap ends at:    1583321400000




In [0]:
coins = coins[4697:24368]

In [12]:
bf.show_time_skips(coins, CURRENCIES)

Nanosec default difference:  60000


In [0]:
coins.set_index(CURRENCIES[0]+' Close Time', inplace=True)

In [0]:
coins.drop([(CURRENCIES[0]+' Open Time'), (CURRENCIES[0]+' Ignore')], axis=1, inplace=True)

In [15]:
coins.head()

,ETHUSDT Open,ETHUSDT High,ETHUSDT Low,ETHUSDT Close,ETHUSDT Volume,ETHUSDT Quote asset volume,ETHUSDT n_trades,ETHUSDT Taker buy base asset volume,ETHUSDT Taker buy quote asset volume
ETHUSDT Close Time,,,,,,,,,
1582133459999,281.50,285.00,281.50,284.00,5363.22231,1.523006e+06,1596,4405.72457,1.251009e+06
1582133519999,284.50,284.50,282.91,283.73,849.05279,2.407136e+05,373,381.29397,1.081573e+05
1582133579999,283.12,283.28,281.64,283.00,731.97512,2.067279e+05,425,313.61358,8.858496e+04
1582133639999,283.00,283.63,282.67,283.39,578.78334,1.639332e+05,353,294.96569,8.356447e+04
1582133699999,283.30,283.69,282.92,283.39,559.30100,1.584016e+05,330,268.61306,7.608924e+04


In [16]:
coins.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19671 entries, 1582133459999 to 1583313659999
Data columns (total 9 columns):
ETHUSDT Open                            19671 non-null float64
ETHUSDT High                            19671 non-null float64
ETHUSDT Low                             19671 non-null float64
ETHUSDT Close                           19671 non-null float64
ETHUSDT Volume                          19671 non-null float64
ETHUSDT Quote asset volume              19671 non-null float64
ETHUSDT n_trades                        19671 non-null int64
ETHUSDT Taker buy base asset volume     19671 non-null float64
ETHUSDT Taker buy quote asset volume    19671 non-null float64
dtypes: float64(8), int64(1)
memory usage: 1.5 MB


In [17]:
coins.index[0]

1582133459999

In [18]:
print(datetime.fromtimestamp(coins.index[0]/1000))
print(datetime.fromtimestamp(coins.index[-1]/1000))


2020-02-19 17:30:59.999000
2020-03-04 09:20:59.999000
